## Calling and assing the data

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/user-click-rata-data/user_click_data.csv
/kaggle/input/user-click-rata-data/newsdata_user_data.csv


In [2]:
user_data = pd.read_csv("/kaggle/input/user-click-rata-data/user_click_data.csv")
data = pd.read_csv("/kaggle/input/user-click-rata-data/newsdata_user_data.csv")

## User Data 

In [3]:
user_data.head()

,user_id,session,article_id,clicked_or_not,time_spent,rating_article
0,1,1,462,0,0.000000,0
1,1,1,2833,0,0.000000,0
2,1,1,1488,1,68.383695,2
3,1,1,461,0,0.000000,0
4,1,1,752,0,0.000000,0


In [4]:
user_data = user_data.drop_duplicates()

In [5]:
user_data.shape

(40759, 6)

In [6]:
table = pd.pivot_table(user_data, values='rating_article', index=['user_id'],columns=['article_id'], aggfunc=np.sum, fill_value=0)

#R_df = user_data.pivot(index = 'user_id', columns ='article_id', values = 'rating_article').fillna(0)
#R_df.head()

In [7]:
table.head()

article_id,0,1,2,3,4,5,6,7,8,9,...,2906,2907,2908,2909,2910,2911,2912,2913,2914,2915
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Cleaning and tokenization of the 'Text'

In [9]:
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random

In [10]:
# Function for removing NonAscii characters
#def _removeNonAscii(s):
#    return "".join(i for i in s if  ord(i)<128)

# Function for converting into lower case
def make_lower_case(text):
    return text.lower()

# Function for removing stop words
def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    texts = [w for w in text if w.isalpha()]
    texts = " ".join(texts)
    return texts

# Function for removing punctuation
def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

# Function for removing the html tags
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

# Applying all the functions in description and storing as a cleaned_desc
#ds['cleaned_desc'] = ds['text'].apply(_removeNonAscii)
data['cleaned_desc'] = data['text'].apply(func = make_lower_case)
data['cleaned_desc'] = data.cleaned_desc.apply(func = remove_stop_words)
data['cleaned_desc'] = data.cleaned_desc.apply(func=remove_punctuation)
data['cleaned_desc'] = data.cleaned_desc.apply(func=remove_html)

## Tfidf vectorizer

In [11]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

## analyzer -- to select individual words# default 
## max_df[0.0,1.0] - used to ignore words with frequency more than 0.8 these words can be useless words as these words may appear only once and may not have a significant meaning
## min_df -- similar reason as the above one. 
## use_idfbool, default=True  -- Enable inverse-document-frequency reweighting.

tf = TfidfVectorizer(analyzer='word',stop_words='english',max_df=0.8,min_df=0.0,use_idf=True,ngram_range=(1,3))
tfidf_matrix = tf.fit_transform(data['cleaned_desc'])

In [12]:
pd.DataFrame(tfidf_matrix.toarray(), columns=tf.get_feature_names())

,aa,aa amazon,aa amazon says,aa batteries,aa batteries instead,aa batteries set,aa compared,aa compared united,aa lithium,aa lithium later,...,überlegungen zur wiedereröffnung,überwiegend,überwiegend um,überwiegend um mehr,𝗺𝗼𝗻𝘁𝗵,𝗺𝗼𝗻𝘁𝗵 pandemic,𝗺𝗼𝗻𝘁𝗵 pandemic sustain,𝗽𝗲𝗿,𝗽𝗲𝗿 𝗺𝗼𝗻𝘁𝗵,𝗽𝗲𝗿 𝗺𝗼𝗻𝘁𝗵 pandemic
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2912,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2913,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2914,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Fining similarities using cosine similarity

In [13]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [14]:
cosine_similarities

array([[1.        , 0.00517514, 0.00600228, ..., 0.00216866, 0.00145762,
        0.00250534],
       [0.00517514, 1.        , 0.03284452, ..., 0.01098935, 0.00852186,
        0.00788426],
       [0.00600228, 0.03284452, 1.        , ..., 0.00769532, 0.00363014,
        0.00536062],
       ...,
       [0.00216866, 0.01098935, 0.00769532, ..., 1.        , 0.00472758,
        0.01107683],
       [0.00145762, 0.00852186, 0.00363014, ..., 0.00472758, 1.        ,
        0.00263229],
       [0.00250534, 0.00788426, 0.00536062, ..., 0.01107683, 0.00263229,
        1.        ]])

## Testing the model for user-1

In [17]:
print(np.dot(table[:1],cosine_similarities))

[[0.57823902 0.80882112 4.8945151  ... 0.49004055 0.38879407 0.23908208]]


In [18]:
recommendation = np.dot(table[:1],cosine_similarities)

In [19]:
similarity_score = list(enumerate(recommendation[0]))

In [23]:
similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)

similarity_score = similarity_score[1:10+1]

In [24]:
news_indices = [i[0] for i in similarity_score]

In [27]:
for i in range(len(news_indices)):
    print("Recommendation "+str(i+1)+" : "+data['title'].iloc[news_indices[i]])
    print()

Recommendation 0 : Sports events hit by the coronavirus epidemic

Recommendation 1 : Italy's top sports body calls for all events to be canceled until April 3

Recommendation 2 : UPDATE 1-Italy's top sports body calls for all events to be cancelled until April 3

Recommendation 3 : Lambda School raises $74M for its virtual coding school where you pay tuition only after you get a job – TechCrunch

Recommendation 4 : COVID SCIENCE-Modern nursing homes safer in pandemic; virus level in nose, throat may help guide treatment

Recommendation 5 : In China, newly-listed ChiNext shares surge in historic reform

Recommendation 6 : Facebook launches climate science info center amid fake news criticism

Recommendation 7 : Another resignation from Brazil's economy ministry

Recommendation 8 : Breakingviews - Corona Capital: Hockey, HBO, Minority businesses

Recommendation 9 : German economy likely to shrink less this year than during 2009 crisis - IfW

